In [1]:
# Libraries to manage data
import numpy as np
import pandas as pd

# Libraries for reqs and scrapping
import requests
from bs4 import BeautifulSoup

In [2]:
# Raw html response
covid_url = 'https://www.worldometers.info/coronavirus/'
req = requests.get(covid_url)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# From table get what we want
# html tags: 
# variables names --> 'thead' --> 'tr' --> 'th'
# variables names --> 'tbody' --> 'tr' --> 'td'
variables = []
variables_html = soup.find_all('thead')[-1].find_all('tr')
for tag in variables_html:
    ths = tag.find_all(['th'])
    row = [th.text for th in ths]
    variables.append(row)
# variables

samples = []
samples_html = soup.find_all('tbody')[0].find_all('tr')
for tr in samples_html:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    samples.append(row)
# samples

df = pd.DataFrame(samples[:len(samples)-6], columns=variables[0]).drop(columns=["#"])       
df.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,\nNorth America\n,"19,899,064","+139,656","460,316","+3,219","12,105,441","+107,151","7,333,307","33,624",,,,,,North America,\n,,
1,\nAsia\n,"19,388,890","+102,840","316,565","+1,325","17,593,324","+103,811","1,479,001","27,528",,,,,,Asia,\n,,
2,\nSouth America\n,"12,093,641","+14,295","342,728",+230,"10,715,935","+1,943","1,034,978","16,591",,,,,,South America,\n,,
3,\nEurope\n,"20,430,639","+225,684","473,306","+5,460","9,671,804","+145,765","10,285,529","26,786",,,,,,Europe,\n,,
4,\nAfrica\n,"2,439,790","+17,157","57,431",+263,"2,064,370","+17,322","317,989","2,709",,,,,,Africa,\n,,


In [4]:
df = df.iloc[8:,:-3].reset_index(drop=True)

In [6]:
# rename columns
df.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df = df[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df.sample(5)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
191,Seychelles,"52,724","5,200",Africa,"98,626",202,,,,184,,18,,"2,048",
92,Norway,"459,636","2,500,706",Europe,"5,440,623","42,077",+225,402,+7,"34,782",,"6,893",31,"7,734",74
49,Dominican Republic,"73,551","801,531",North America,"10,897,555","156,585",+788,"2,372",+5,"121,988",+665,"32,225",235,"14,369",218
174,Taiwan,"4,910","117,034",Asia,"23,836,581",749,+7,7,,612,+1,130,,31,0.3
157,Guinea-Bissau,"17,263","34,336",Africa,"1,988,971","2,447",,44,,"2,378",,25,5,"1,230",22


In [14]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'
# addding 'WHO Region' column
df['WHO Region'] = df['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df[df['WHO Region'].isna()]['Country/Region'].unique()
df.tail()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
209,Solomon Islands,"6,480",4500,Australia/Oceania,694472,17,NaN,NaN,NaN,5,NaN,12,NaN,24,NaN,Western Pacific
210,Saint Pierre Miquelon,"509,600",2946,NorthAmerica,5781,14,NaN,NaN,NaN,14,NaN,0,NaN,2422,NaN,NaN
211,Montserrat,"125,376",626,NorthAmerica,4993,13,NaN,1,NaN,12,NaN,0,NaN,2604,200,NaN
212,Western Sahara,NaN,NaN,Africa,603954,10,NaN,1,NaN,8,NaN,1,NaN,17,2,Africa
213,Anguilla,"175,982",2651,NorthAmerica,15064,10,NaN,NaN,NaN,4,NaN,6,NaN,664,NaN,NaN


In [15]:
# fix data
for col in df.columns[2:]:
    # replace comma with empty string
    df[col] = df[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df[col] = df[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df = df.replace('', np.nan)
df.sample(5)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
165,Liberia,"7,704",39377,Africa,5111280,1773,NaN,83,NaN,1406,NaN,284,2,347,16,Africa
122,Cuba,"113,321",1283188,NorthAmerica,11323433,9671,83,137,NaN,8658,66,876,9,854,12,Americas
163,San Marino,"662,496",22501,Europe,33964,2025,43,54,2,1694,9,277,5,59622,1590,Europe
152,Curaçao,"177,681",29211,NorthAmerica,164401,3699,NaN,11,NaN,1889,NaN,1799,4,22500,67,NaN
25,Bangladesh,"18,270",3022537,Asia,165438217,495841,1632,7156,27,429351,2622,59334,NaN,2997,43,South-EastAsia


In [17]:
# save as .csv file
df.to_csv('covid19_data.csv', index=False)